In [ ]:
import numpy as np
import torch
import math

big_array = np.random.randn(1000, 237, 512, 512).astype(np.float32)
#torch_arr = torch.from_numpy(big_array)
#torch_arr.share_memory_()


class DataLoaderTest(torch.utils.data.IterableDataset):
    def __init__(
            self, 
            data, shuffle_data=False
        ):
        super().__init__()
        self.data = data
        self.shuffle_data = shuffle_data

    def __iter__(self):
        if self.shuffle_data:
            self.data = shuffle(self.data)
        
        for data in zip(self.data):
            yield data
            
# Define a `worker_init_fn` that configures each dataset copy differently
def worker_init_fn(worker_id):
    worker_info = torch.utils.data.get_worker_info()
    dataset = worker_info.dataset
    overall_start = 0
    overall_end = len(dataset.data)
    # configure the dataset to only process the split workload
    per_worker = int(math.ceil((overall_end - overall_start) / float(worker_info.num_workers)))
    worker_id = worker_info.id
    start = overall_start + worker_id * per_worker
    end = min(start + per_worker, overall_end)
    dataset.dataset = dataset.data[start:end]

In [ ]:
dataset_train = DataLoaderTest(big_array)

train_loader = torch.utils.data.DataLoader(
    dataset_train, batch_size=2, 
    num_workers=10, pin_memory=False, prefetch_factor=2,
#    worker_init_fn=worker_init_fn, drop_last=True
)

In [ ]:
def run():
    for in_x in train_loader:
        pass

In [ ]:
%load_ext memory_profiler

In [ ]:
%memit run()

In [ ]:
import os,psutil
pid = os.getpid()
print(pid)

In [ ]:
ps = psutil.Process(pid)
memory_use = ps.memory_info()
print(memory_use.rss // (8 * 1024 * 1024)) # Gb

In [ ]:
# shared    - 600 
# no shared - 401

In [ ]:
existing_shm = shared_memory.SharedMemory(name='psm_ea2f5eac')
c_list = []
for i in range(1_000_000):
    c_list.append(np.ndarray((10, 237, 512, 512), dtype=np.float32, buffer=existing_shm.buf))